In [1]:
import numpy as np
import tensorflow as tf
import random
import numpy as np
from collections import deque

import gym
env = gym.make('CartPole-v0')

# Constants defining our neural network
input_size = env.observation_space.shape[0]
output_size = env.action_space.n

#dis = 0.9
#REPLAY_MEMORY = 50000

# 리플레이를 저장할 리스트
REPLAY_MEMORY = deque()

# 하이퍼파라미터
INITIAL_EPSILON = 0.5
learning_rate = 0.0001
max_episodes = 10000
discount_factor = 0.9
episode_list = []
train_error_list = []
actions_list = []

# 테스트 에피소드 주기
TEST_PERIOD = 100

# src model에서 target model로 trainable variable copy 주기
COPY_PERIOD = 10

[2017-11-14 15:40:58,436] Making new env: CartPole-v0


In [2]:
class DQN:
    def __init__(self, session, input_size, output_size, name="main"):
        # 네트워크 정보 입력
        self.session = session
        self.input_size = input_size
        self.output_size = output_size
        self.net_name = name
        self.hidden_size = 32 ##
        # 네트워크 생성
        self._build_network()
        
    def _build_network(self) :#, h_size = 10, l_rate = 1e-1):
        with tf.variable_scope(self.net_name):
            #self._X = tf.placeholder(
             #   tf.float32, [None, self.input_size], name = "input_x")
        
            # First layer of weights
            #W1 = tf.get_variable("W1", shape=[self.input_size, h_size],
            #                 initializer=tf.contrib.layers.xavier_initializer())
            #layer1 = tf.nn.tanh(tf.matmul(self._X, W1))
        
            # Second Layer of weights
            #W2 = tf.get_variable("W2", shape=[h_size, self.output_size],
            #                 initializer=tf.contrib.layers.xavier_initializer())
        
            # Q prediction
            #self._Qpred = tf.matmul(layer1, W2)
            
            # Vanilla Neural Network (Just one hidden layer)
            self.X = tf.placeholder(shape=[None, self.input_size], dtype=tf.float32)
            self.Y = tf.placeholder(shape=[None], dtype=tf.float32)

            self.W1 = tf.Variable(tf.truncated_normal(shape=[self.input_size, self.hidden_size], mean=0.0, stddev=1.0))
            self.B1 = tf.Variable(tf.zeros(shape=[self.hidden_size]))
            self.W2 = tf.Variable(tf.truncated_normal(shape=[self.hidden_size, self.output_size], mean=0.0, stddev=1.0))
            self.B2 = tf.Variable(tf.zeros(shape=[self.output_size]))

            self.L1 = tf.nn.relu(tf.matmul(self.X, self.W1) + self.B1)

            self.Qpred = tf.matmul(self.L1, self.W2) + self.B2
        
        # We need to define the parts of the network needed for learning a policy
        #self._Y = tf.placeholder(
        #    shape = [None, self.output_size], dtype = tf.float32)
    
        # Loss function
        #self._loss = tf.reduce_mean(tf.square(self._Y - self._Qpred))
        # Learning
        #self._train = tf.train.AdamOptimizer(
        #    learning_rate=l_rate).minimize(self._loss)
         # 손실 함수 및 최적화 함수
        self.action = tf.placeholder(shape=[None, self.output_size], dtype=tf.float32)
        Q_action = tf.reduce_sum(tf.multiply(self.Qpred, self.action), reduction_indices=1)
        self.loss = tf.reduce_mean(tf.square(self.Y - Q_action))
        self.optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(self.loss)

    
    def predict(self, state):
        x = np.reshape(state, newshape = [1, self.input_size])
        return self.session.run(self._Qpred, feed_dict = {self._X: x})
    
    #def update(self, x_stack, y_stack):
    #    return self.session.run([self._loss, self._train], feed_dict = 
    #                           {self._X: x_stack, self._Y: y_stack})
     # e-greedy 를 사용하여 action값 구함
    def egreedy_action(self, epsilon, env, state):
        if np.random.rand(1) < epsilon:
            action = env.action_space.sample()
            #print("Episode: {0}, Action: {1}".format(episode, action))
        else:
            Q_h = self.predict(state)
            action = np.argmax(Q_h)
            #print("Episode: {0}, State: {1}, Q_h: {2}, Action: {3}".format(episode, state, Q_h, action))
        return action

In [3]:
def update_from_memory(mainDQN, targetDQN, batch_size):
    state_batch = np.ndarray(shape=[batch_size, mainDQN.input_size])
    action_batch = np.ndarray(shape=[batch_size, mainDQN.output_size])

    minibatch = random.sample(REPLAY_MEMORY, batch_size)
    i = 0
    y_batch = []
    for sample in minibatch:
        state, action, reward, new_state, done = sample         # unpacking

        if done:
            y_batch.append(reward)
        else:
            y_batch.append(reward + discount_factor * np.max(targetDQN.predict(onehot(targetDQN.input_size, new_state))))

        one_hot_action = np.zeros(mainDQN.output_size) # [0.0, 0.0]
        one_hot_action[action] = 1

        state_batch[i] = state
        action_batch[i] = one_hot_action
        i += 1

    # DQN 알고리즘으로 학습
    loss_value, _ = mainDQN.session.run([mainDQN.loss, mainDQN.optimizer],
                                        feed_dict={mainDQN.X: state_batch, mainDQN.Y: y_batch, mainDQN.action: action_batch})
    return loss_value


In [4]:
#def replay_train(mainDQN, targetDQN, train_batch):
#    x_stack = np.empty(0).reshape(0, input_size)
#    y_stack = np.empty(0).reshape(0, output_size)
#    
#    # Get stored information from the buffer
#    for state, action, reward, next_state, done in train_batch:
#        Q = mainDQN.predict(state)
#        
#        # terminal?
#        if done:
#            Q[0, action] = reward
#        else:
#            # get target from target DQN (Q')
#            Q[0, action] = reward + dis * np.max(targetDQN.predict(next_state))
#            
#        y_stack = np.vstack([y_stack, Q])
#        x_stack = np.vstack([x_stack, state])
#        
#    # Train our network using target and predicted Q values on each episode
#    return mainDQN.update(x_stack, y_stack)

In [5]:
#def get_copy_var_ops(*, dest_scope_name = "target", src_scope_name = "main"):
#    # Copy variables src_scope to dest_scope
#    op_holder = []
#    
#    src_vars = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope = src_scope_name)
#    dest_vars = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope = dest_scope_name)
#    
#    for src_var, dest_var in zip(src_vars, dest_vars):
#        op_holder.append(dest_var.assign(src_var.value()))
#        
#    return op_holder

In [4]:
#def bot_play(mainDQN):
#    # See our trained network in action
#    s = env.reset()
#    reward_sum = 0
#    while True:
#        env.render()
#        a = np.argmax(mainDQN.predict(s))
#        s, reward, done, _ = env.step(a)
#        reward_sum += reward
#        if done:
#            print("Total score: {}".format(reward_sum))
#            break

def bot_play(DQN, env, isLast):
    """
    See our trained network in action
    """
    state = env.reset()
    reward_sum = 0
    done = False
    num_action = 0
    while not done and num_action < 1000:
        if isLast: env.render()
        action = np.argmax(DQN.predict(onehot(env.observation_space.n, state)))
        new_state, reward, done, info = env.step(action)
        reward_sum += reward
        state = new_state
        num_action += 1
        if done or num_action >= 1000:
            print("Done!, {0}, {1}".format(num_action, reward_sum))

    return reward_sum


In [5]:
def draw_error_values():
    fig = plt.figure(figsize=(20, 5))
    plt.ion()
    ax1 = plt.subplot(111)
    ax2 = ax1.twinx()
    ax1.plot(episode_list[0:], train_error_list[0:], 'r', label='Train Error Values')
    ax2.plot(episode_list[0:], actions_list[0:], 'b', label='Number of Actions')
    ax1.set_ylabel('Train Error Values', color='r')
    ax2.set_ylabel('Number of Actions', color='b')
    ax1.set_xlabel('Episodes')
    plt.grid(True)
    plt.legend(loc='upper right')
    plt.show()
    input("Press Enter to close the trained error figure...")
    plt.close(fig)


In [6]:
def get_copy_var_ops(src_scope_name='main', target_scope_name='target'):
    op_holder = []

    src_vars = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope=src_scope_name)
    target_vars = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope=target_scope_name)

    for src_var, target_var in zip(src_vars, target_vars):
        op_holder.append(target_var.assign(src_var.value()))

    return op_holder


In [7]:
def saveModel(session, src_scope_name='main', path='./cartpole.ckpt'):
    src_vars = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope=src_scope_name)
    tf.train.Saver(src_vars).save(session, path)
    print("Model saved successfully!")

def onehot(input_size, state):
    return np.eye(input_size)[state]

In [8]:
if __name__ == "__main__":
    #env = gym.make("FrozenLake-v3")
    #input_size = env.observation_space.n     # 4  action (키보드)
    #output_size = env.action_space.n         # 2

    # 미니배치 - 꺼내서 사용할 리플레이 갯수
    BATCH_SIZE = 32

    with tf.Session() as sess:
        # DQN 클래스의 mainDQN 인스턴스 생성
        mainDQN = DQN(sess, input_size, output_size, name='main')
        targetDQN = DQN(sess, input_size, output_size, name='target')

        # 변수 초기화
        init = tf.global_variables_initializer()
        sess.run(init)
        epsilon = INITIAL_EPSILON

        copy_ops = get_copy_var_ops(src_scope_name='main', target_scope_name='target')
        sess.run(copy_ops)

        state = env.reset()
        env.render()
        for episode in range(max_episodes):
            state = env.reset()
            rAll = 0
            done = False
            epsilon *= 0.999

            while not done:
                #env.render()
                # action을 수행함 --> Get new state and reward from environment
                action = mainDQN.egreedy_action(epsilon, env, onehot(input_size, state))
                new_state, reward, done, _ = env.step(action)

                # state, action, reward, next_state, done 을 메모리에 저장
                REPLAY_MEMORY.append((onehot(input_size, state), action, reward, new_state, done))

                # 메모리에 10000개 이상의 값이 들어가면 가장 먼저 들어간 것부터 삭제
                if len(REPLAY_MEMORY) > 10000:
                    REPLAY_MEMORY.popleft()

                # REPLAY_MEMORY 크기가 BATCH_SIZE 보다 크면 학습
                if len(REPLAY_MEMORY) > BATCH_SIZE:
                    mean_loss_value = update_from_memory(mainDQN, targetDQN, BATCH_SIZE)

                rAll += reward
                state = new_state

            if len(REPLAY_MEMORY) > BATCH_SIZE:
                episode_list.append(episode)
                train_error_list.append(mean_loss_value)
                actions_list.append(rAll)

            if episode != 0 and episode % COPY_PERIOD == 0:
                a = sess.run(copy_ops)

            if episode % TEST_PERIOD == 0:
                total_reward = 0
                for i in range(10):
                    total_reward += bot_play(mainDQN, env, False)

                ave_reward = total_reward / 10
                print("episode: {0}, Epsilon: {1}, Evaluation Average Reward: {2}".format(episode,epsilon, ave_reward))
                if ave_reward >= 1.0:
                    break

        saveModel(sess, src_scope_name='main', path='./cartpole.ckpt')

        env.reset()
        env.close()
        draw_error_values()

        input("Press Enter to make the trained bot play...")
        bot_play(mainDQN, env, True)


IndexError: arrays used as indices must be of integer (or boolean) type

In [9]:
def main():
    max_episodes = 5000
    # store the previous observations in replay memory
    replay_buffer = deque()
    
    with tf.Session() as sess:
        mainDQN = DQN(sess, input_size, output_size, name = "main")
        targetDQN = DQN(sess, input_size, output_size, name = "target")
        tf.global_variables_initializer().run()
    
        # initial copy q_net -> target_net
        copy_ops = get_copy_var_ops(dest_scope_name = "target", src_scope_name = "main")
    
        sess.run(copy_ops)
    
        for episode in range(max_episodes):
            e = 1. / ((episode / 10) + 1)
            done = False
            step_count = 0
            state = env.reset()
        
            while not done:
                if np.random.rand(1) < e:
                    action = env.action_space.sample()
                else:
                    # Choose an action by greedily from the Q-netsork
                    action = np.argmax(mainDQN.predict(state))
                
                # Get new state and reward from environment
                next_state, reward, done, _ = env.step(action)
                if done: # Penalty
                    reward = -100
                
                # Save the experience to our buffer
                replay_buffer.append((state, action, reward, next_state, done))
                if len(replay_buffer) > REPLAY_MEMORY:
                    replay_buffer.popleft()

                state = next_state
                step_count += 1
                if step_count > 10000: # Good enough. Let's move on
                    break

            print("Episode: {}  step:  {}".format(episode, step_count))
            if step_count > 10000:
                pass
                # break

            if episode % 10 == 1:  # train every 10 episode
                # Get a random batch of experiences.
                for _ in range(50):
                    minibatch = random.sample(replay_buffer, 10)
                    loss, _ = replay_train(mainDQN, targetDQN, minibatch)

                print("Loss: ", loss)
                #copy q_net -> target_net
                sess.run(copy_ops)

        bot_play(mainDQN)
            
if __name__ == "__main__":
    main()

TypeError: get_copy_var_ops() got an unexpected keyword argument 'dest_scope_name'

In [8]:
bot_play(mainDQN)

RuntimeError: Attempted to use a closed Session.

In [1]:
a = 1
print(type(a))
def sum() :
    global a
    a = 0.1
    print(type(a))
    print(a)
sum()
print(type(a))
print(a)

<class 'int'>
<class 'float'>
0.1
<class 'float'>
0.1
